In [ ]:
## Homework 3B – AHP Decision Making

This notebook applies the Analytic Hierarchy Process (AHP) to rank three school alternatives (A, B, C) across six criteria:

- learning  
- college_prep  
- friends  
- music_classes  
- vocational_trng  
- school_life  

---


In [11]:
# ——— Cell: AHP Analysis ——————————————————————————————————————————————
import pandas as pd
import numpy as np
import networkx as nx
import ahpy

# 1) load your Excel file (must be in the same folder as this notebook)
xls = 'comparisons.xlsx'

# 2) read each sheet as an adjacency matrix
pairwise_learning        = pd.read_excel(xls, sheet_name='learning',        index_col=0)
pairwise_college_prep    = pd.read_excel(xls, sheet_name='college_prep',    index_col=0)
pairwise_friends         = pd.read_excel(xls, sheet_name='friends',         index_col=0)
pairwise_music_classes   = pd.read_excel(xls, sheet_name='music_classes',   index_col=0)
pairwise_vocational_trng = pd.read_excel(xls, sheet_name='vocational_trng', index_col=0)
pairwise_school_life     = pd.read_excel(xls, sheet_name='school_life',     index_col=0)
pairwise_criteria        = pd.read_excel(xls, sheet_name='criteria',        index_col=0)

# helper to turn an adjacency df into the { (i,j): weight } dict
def to_pairs(df):
    G = nx.from_pandas_adjacency(df, create_using=nx.MultiDiGraph())
    return {
        (u, v): d['weight']
        for u, v, d in G.edges(data=True)
        if np.isfinite(d['weight'])
    }

# 3) build the six Compare objects
learning        = ahpy.Compare('learning',         to_pairs(pairwise_learning),        random_index='saaty')
college_prep    = ahpy.Compare('college_prep',     to_pairs(pairwise_college_prep),    random_index='saaty')
friends         = ahpy.Compare('friends',          to_pairs(pairwise_friends),         random_index='saaty')
music_classes   = ahpy.Compare('music_classes',    to_pairs(pairwise_music_classes),   random_index='saaty')
vocational_trng = ahpy.Compare('vocational_trng',  to_pairs(pairwise_vocational_trng), random_index='saaty')
school_life     = ahpy.Compare('school_life',      to_pairs(pairwise_school_life),     random_index='saaty')

criteria = ahpy.Compare('criteria', to_pairs(pairwise_criteria), random_index='saaty')
criteria.add_children([
    learning,
    college_prep,
    friends,
    music_classes,
    vocational_trng,
    school_life
])

# 4) print exactly the numbers you need
print(">> Criteria global weights:")
for name, w in criteria.global_weights.items():
    print(f"   {name:<15}: {w:.4f}")

print("\n>> Alternative (school) weights:")
for name, w in criteria.target_weights.items():
    print(f"   {name:<3}: {w:.4f}")

print("\n>> Consistency ratios (should all be < 0.1):")
for comp in [learning, college_prep, friends, music_classes, vocational_trng, school_life, criteria]:
    print(f"   {comp.name:<15}: {comp.consistency_ratio:.4f}")


>> Criteria global weights:
   learning       : 0.3208
   college_prep   : 0.2374
   friends        : 0.1395
   music_classes  : 0.1391
   vocational_trng: 0.1285
   school_life    : 0.0348

>> Alternative (school) weights:
   B  : 0.3797
   A  : 0.3301
   C  : 0.2902

>> Consistency ratios (should all be < 0.1):
   learning       : 0.0516
   college_prep   : 0.0516
   friends        : 0.0000
   music_classes  : 0.0825
   vocational_trng: 0.0000
   school_life    : 0.0000
   criteria       : 0.2272


In [ ]:
## Results & Interpretation

### 1. Criteria Importance
- **Learning** (0.3208) and **College Preparation** (0.2374) dominate, together accounting for ~56% of the decision weight.  
- **Friends** (0.1395) and **Music Classes** (0.1391) are next in importance, each around 14%.  
- **Vocational Training** (0.1285) carries about 13% weight.  
- **School Life** (0.0348) is least important (≈3.5%).

### 2. School Rankings
Based on the weighted sum of how each school performs under those six criteria:
1. **School B**: 37.97%  
2. **School A**: 33.01%  
3. **School C**: 29.02%  

This suggests **School B** is the top choice, followed by **A**, then **C**.

### 3. Consistency Check
- All individual‐criterion matrices are well within Saaty’s 0.10 threshold (CR ≤ 0.0825), indicating reliable pairwise judgments.  
- The overall **criteria** comparison still has a CR of 0.2272, which exceeds 0.10. You may wish to revisit those six criteria judgments to improve consistency.  

---

> **Conclusion:**  
> After refining any inconsistent judgments in the top‐level criteria, School B will remain the recommended option based on this AHP analysis.  
